[20220924]

# 2. Model 6 - 2

continue train the model 6

After Preparation ready. Build and train the model in here. its using LSTM one layer. with data gen4

## Goal
Find patterns on the features for target

## Plan
Checklist what will be done on the notebook :

        [ ] Get Data
        [ ] Modelling
        [ ] Evaluation

In [13]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
from statsmodels.tsa.arima.model import ARIMA
import sys
sys.path.insert(1,'../../src/')

## import myfunc inside src folder
from ramal import eval_model, jampi

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import metrics, optimizers
import keras
import tensorflow as tf
from keras.models import load_model

from sklearn.model_selection import train_test_split

In [15]:
with open("../../config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

PATH_RAW = config['path']['raw']
PATH_INTERIM = config['path']['interim']
PATH_PROCESSED = config['path']['processed']
# PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'
PATH_MODEL = config['path']['artifact']+'iteration_3'+'/'
PATH_UDF = config['path']['udf']

## Get Data

In [16]:
Xt, yt, Xo, yo = joblib.load(PATH_INTERIM+'feat_target_gen4_iter3.pkl')
Xt = Xt.astype("float32")
yt = yt.astype("float32")

Xo = Xo.astype("float32")
yo = yo.astype("float32")

In [17]:
Xt.shape

(409713, 15, 2)

In [18]:
yt.shape

(409713, 1)

In [19]:
SEQ = Xt.shape[1]

In [20]:
## not the best way to split time series dataset
x_tr, x_test, y_tr, y_test = train_test_split(
    Xt, yt, test_size=0.25, random_state=42)

x_train, x_val, y_train, y_val = train_test_split(
    x_tr, y_tr, test_size=0.2, random_state=42)

## Model 1

In [18]:
# os.environ["CUDA_VISIBLE_DEVICES"] 

In [19]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [20]:
# depd = {'auc_1' : metrics.AUC}
mod_ts = load_model(PATH_MODEL + 'ws_data_model6_iter3.h5',compile=False)

In [8]:
early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',mode='min', patience=6) 

check_p = keras.callbacks.ModelCheckpoint(
        filepath=PATH_MODEL+'ws_data_model6_iter3-2.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True)

opt = tf.keras.optimizers.Adam(learning_rate=0.00015)

# mod_ts.compile(loss='mean_squared_error', optimizer=opt, metrics=[
#         tf.keras.metrics.RootMeanSquaredError(name='the_rmse'),
#         tf.keras.metrics.MeanAbsolutePercentageError(name='the_mape'),
#     ])

In [22]:
history = mod_ts.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=50,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
    callbacks=[early_stop,check_p]
)

Epoch 1/50
1921/1921 [==============================] - 109s 54ms/step - loss: 0.4183 - the_rmse: 0.6468 - the_mape: 113384.9531 - val_loss: 0.4801 - val_the_rmse: 0.6929 - val_the_mape: 166200.5469
Epoch 2/50
1921/1921 [==============================] - 108s 56ms/step - loss: 0.4156 - the_rmse: 0.6447 - the_mape: 102206.2812 - val_loss: 0.4757 - val_the_rmse: 0.6897 - val_the_mape: 185715.0781
Epoch 3/50
1921/1921 [==============================] - 107s 56ms/step - loss: 0.4133 - the_rmse: 0.6429 - the_mape: 116581.8828 - val_loss: 0.4775 - val_the_rmse: 0.6910 - val_the_mape: 188219.7812
Epoch 4/50
1921/1921 [==============================] - 107s 56ms/step - loss: 0.4116 - the_rmse: 0.6415 - the_mape: 115109.4375 - val_loss: 0.4791 - val_the_rmse: 0.6922 - val_the_mape: 224298.3594
Epoch 5/50
1921/1921 [==============================] - 108s 56ms/step - loss: 0.4099 - the_rmse: 0.6402 - the_mape: 120537.5547 - val_loss: 0.4749 - val_the_rmse: 0.6892 - val_the_mape: 150942.3281
Epoch

## Eval

In [14]:
df_epoch_hist = pd.DataFrame(history.history)
df_epoch_hist[['loss','val_loss']].plot()

NameError: name 'history' is not defined

In [9]:
# depd = {'auc_1' : metrics.AUC}
model_load = load_model(PATH_MODEL + 'ws_data_model6_iter3-2.h5',compile=False)

In [10]:
model_load.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 128)           67072     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 119,105
Trainable params: 119,105
Non-trainable params: 0
_________________________________________________________________


In [21]:
model_load.compile(loss='mean_squared_error', optimizer=opt, metrics=[
        tf.keras.metrics.RootMeanSquaredError(name='the_rmse'),
        tf.keras.metrics.MeanAbsolutePercentageError(name='the_mape'),
    ])

In [22]:
model_load.evaluate(x_test,y_test)

3201/3201 [==============================] - 33s 10ms/step - loss: 0.4774 - the_rmse: 0.6909 - the_mape: 123054.2734


[0.47740721702575684, 0.690946638584137, 123054.2734375]

In [23]:
model_load.evaluate(Xo,yo)

521/521 [==============================] - 6s 11ms/step - loss: 0.4681 - the_rmse: 0.6842 - the_mape: 103479.2656


[0.46813929080963135, 0.6842070817947388, 103479.265625]